In [1]:
import sys
sys.version

'3.13.6 (tags/v3.13.6:4e66535, Aug  6 2025, 14:36:00) [MSC v.1944 64 bit (AMD64)]'

In [2]:
!pip install tensorflow


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
import tensorflow as tf
print(tf.__version__)

2.20.0


In [4]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [5]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ANUSTUPDAS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANUSTUPDAS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_csv("Spotify_Song_Dataset.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
df.columns

Index(['artist', 'song', 'link', 'text'], dtype='object')

In [8]:
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = text.split()   # 🔥 safer than word_tokenize
    tokens = [word for word in tokens if word not in stop_words]
    return " ".join(tokens)

In [9]:
df['cleaned_text'] = df['text'].apply(preprocess_text)
df[['text', 'cleaned_text']].head()

,text,cleaned_text
0,"Look at her face, it's a wonderful face \nAnd...",look face wonderful face means something speci...
1,"Take it easy with me, please \nTouch me gentl...",take easy please touch gently like summer even...
2,I'll never know why I had to go \nWhy I had t...,ill never know go put lousy rotten show boy to...
3,Making somebody happy is a question of give an...,making somebody happy question give take learn...
4,Making somebody happy is a question of give an...,making somebody happy question give take learn...


In [10]:
df['label'] = df['song'] + " - " + df['artist']

In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=10000)

X = tfidf.fit_transform(df['cleaned_text']).toarray()
y = df['label_encoded']

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(df['label_encoded'].unique()), activation='softmax')
])

C:\Users\ANUSTUPDAS\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/10
649/649 ━━━━━━━━━━━━━━━━━━━━ 194s 294ms/step - accuracy: 0.0000e+00 - loss: 11.0440 - val_accuracy: 0.0000e+00 - val_loss: 11.1898
Epoch 2/10
649/649 ━━━━━━━━━━━━━━━━━━━━ 185s 285ms/step - accuracy: 0.0000e+00 - loss: 10.8574 - val_accuracy: 0.0000e+00 - val_loss: 11.6087
Epoch 3/10
649/649 ━━━━━━━━━━━━━━━━━━━━ 191s 295ms/step - accuracy: 7.2275e-05 - loss: 10.7741 - val_accuracy: 0.0000e+00 - val_loss: 11.9734
Epoch 4/10
649/649 ━━━━━━━━━━━━━━━━━━━━ 190s 293ms/step - accuracy: 2.1683e-04 - loss: 10.5669 - val_accuracy: 0.0000e+00 - val_loss: 12.2869
Epoch 5/10
649/649 ━━━━━━━━━━━━━━━━━━━━ 181s 278ms/step - accuracy: 0.0016 - loss: 9.9798 - val_accuracy: 0.0000e+00 - val_loss: 12.6962
Epoch 6/10
649/649 ━━━━━━━━━━━━━━━━━━━━ 194s 300ms/step - accuracy: 0.0082 - loss: 9.0452 - val_accuracy: 0.0000e+00 - val_loss: 13.3109
Epoch 7/10
649/649 ━━━━━━━━━━━━━━━━━━━━ 192s 295ms/step - accuracy: 0.0300 - loss: 7.8791 - val_accuracy: 0.0000e+00 - val_loss: 14.1507
Epoch 8/10
649/649 ━━

In [17]:
def predict_song_artist(input_text):
    cleaned = preprocess_text(input_text)
    vector = tfidf.transform([cleaned]).toarray()
    
    prediction = model.predict(vector)
    predicted_index = prediction.argmax()
    
    return label_encoder.inverse_transform([predicted_index])[0]

In [20]:
test_lyrics = "Take it easy with me, please"

prediction = predict_song_artist(test_lyrics)
print("Predicted Song & Artist:", prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Song & Artist: It's So Easy (To Fall In Love) - Linda Ronstadt
